In [3]:
#Load dependencies
from datetime import datetime as dt, timedelta
import pandas as pd
import numpy as np
from uszipcode import SearchEngine

In [2]:
#pip install uszipcode

  Created wheel for autopep8: filename=autopep8-1.5-py2.py3-none-any.whl size=43272 sha256=9b7a0f6985f0652962af60e6944aa00e1defa123c8eac901179ea299ba6be905
  Stored in directory: C:\Users\ruby\AppData\Local\pip\Cache\wheels\2b\dc\d5\e4bc37df332b7a88022bc79d6cdccb79068fdc1620d86afc3a
Successfully built autopep8
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Define data files to import
url2017='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-2017.txt'
url2018='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-2018.txt'
url2019='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-2019.txt'
url2020='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-monthly.txt'
nullzip=pd.read_csv('../Clean Data Files/311latlngzipcodes.csv',dtype={'calczip':str})

#Define dataframe column names and select numeric and date columns
cols=['case number','sr location','county','district','neighborhood','tax id','trash quad','recycle quad','trash day','heavy trash day','recycle day','key map',
      'management district','department','division','sr type','queue','sla','status','sr create date','due date','date closed','overdue','title','x','y','latitude',
      'longitude','channel type']
numcols=['latitude','longitude']
datecols=['sr create date','due date','date closed']

#Create zipcode retrieval function
search=SearchEngine(simple_zipcode=False)
def zipinfo(lat,lng):
    zipdata=search.by_coordinates(lat,lng,radius=3,returns=1)
    for zipcode in zipdata:
        return zipcode.zipcode

In [5]:
#Create 2017 data frames
data2017=pd.read_csv(url2017,header=5,sep='|',error_bad_lines=False)
data2017=data2017.drop(data2017.index[0]).reset_index(drop=True)
data2017.columns=cols
data2017[cols]=data2017[cols].apply(lambda x:x.str.strip()).replace(r'^\s*$',np.nan,regex=True)
data2017[datecols]=data2017[datecols].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')
data2017[numcols]=data2017[numcols].apply(pd.to_numeric,errors='coerce')
top2017=data2017['sr type'].value_counts()[lambda x:x>=10000].index.tolist()
data2017.shape

b'Skipping line 9979: expected 29 fields, saw 30\nSkipping line 16339: expected 29 fields, saw 30\n'
b'Skipping line 211068: expected 29 fields, saw 30\n'
b'Skipping line 294299: expected 29 fields, saw 30\n'
b'Skipping line 327924: expected 29 fields, saw 30\n'


(364664, 29)

In [6]:
#Create 2018 data frames
data2018=pd.read_csv(url2018,header=5,sep='|',error_bad_lines=False)
data2018=data2018.drop(data2018.index[0]).reset_index(drop=True)
data2018.columns=cols
data2018[cols]=data2018[cols].apply(lambda x:x.str.strip()).replace(r'^\s*$',np.nan,regex=True)
data2018[datecols]=data2018[datecols].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')
data2018[numcols]=data2018[numcols].apply(pd.to_numeric,errors='coerce')
top2018=data2018['sr type'].value_counts()[lambda x:x>=10000].index.tolist()
data2018.shape

b'Skipping line 124864: expected 29 fields, saw 30\n'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(399953, 29)

In [7]:
#Create 2019 data frames
data2019=pd.read_csv(url2019,header=5,sep='|',error_bad_lines=False)
data2019=data2019.drop(data2019.index[0]).reset_index(drop=True)
data2019.columns=cols
data2019[cols]=data2019[cols].apply(lambda x:x.str.strip()).replace(r'^\s*$',np.nan,regex=True)
data2019[datecols]=data2019[datecols].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')
data2019[numcols]=data2019[numcols].apply(pd.to_numeric,errors='coerce')
top2019=data2019['sr type'].value_counts()[lambda x:x>=10000].index.tolist()
data2019.shape

b'Skipping line 86859: expected 29 fields, saw 31\n'
b'Skipping line 124913: expected 29 fields, saw 30\n'
b'Skipping line 144497: expected 29 fields, saw 30\n'
b'Skipping line 218652: expected 29 fields, saw 31\n'
b'Skipping line 349873: expected 29 fields, saw 30\n'


(395258, 29)

In [10]:
#Create all complete years dataframe
tempdata311=data2017.append([data2018,data2019])
data311=pd.merge(tempdata311,nullzip,on=['latitude','longitude'],how='left')
data311['year']=data311['sr create date'].dt.strftime('%Y')
data311['month']=data311['sr create date'].dt.strftime('%m')
data311=data311[pd.notnull(data311['latitude'])]
data311['truezip']='77'+data311['sr location'].str.extract(r'77(\d{3}\-?\d{0,4})')
data311['zipcode']=np.where(data311['truezip'].isnull()==True,data311['calczip'],data311['truezip'])
data311['openclosetime']=data311['date closed']-data311['sr create date']
data311['daystoclose']=data311['openclosetime']/timedelta(days=1)
data311['openduetime']=data311['due date']-data311['sr create date']
data311['daysdue']=data311['openduetime']/timedelta(days=1)
data311['missedduedate']=np.where(data311['due date']>data311['date closed'],0,1)
types311=data311.groupby(['sr type','year'])['case number'].count().unstack('year').reset_index()
types311.columns=['sr type','2017','2018','2019']
data311.shape

(1059994, 39)

In [9]:
#Create 2020 data frames
data2020=pd.read_csv(url2020,header=5,sep='|',error_bad_lines=False)
data2020=data2020.drop(data2020.index[0]).reset_index(drop=True)
data2020.columns=cols
data2020[cols]=data2020[cols].apply(lambda x:x.str.strip()).replace(r'^\s*$',np.nan,regex=True)
data2020[datecols]=data2020[datecols].apply(pd.to_datetime,errors='coerce')
data2020[numcols]=data2020[numcols].apply(pd.to_numeric,errors='coerce')
data2020=data2020[pd.notnull(data2020['latitude'])]
data2020['zipcode']='77'+data2020['sr location'].str.extract(r'77(\d{3}\-?\d{0,4})')
data2020['openclosetime']=data2020['date closed']-data2020['sr create date']
data2020['openduetime']=data2020['due date']-data2020['sr create date']
top2020=data2020['sr type'].value_counts()[lambda x:x>=10000].index.tolist()
data2020.shape

(25328, 32)

In [11]:
#Create csv of number of service requests by type per year
types311.to_csv('../Clean Data Files/Houston 311 SR Types by Year.csv',index=False,header=True)

In [12]:
#Create dataframe containing service requests with around 10000 a year
toprequests=sorted(np.unique(top2017+top2018+top2019+top2020))
topdata=data311.loc[data311['sr type'].isin(toprequests)].reset_index()
topdata

,index,case number,sr location,county,district,neighborhood,tax id,trash quad,recycle quad,trash day,...,calczip,year,month,truezip,zipcode,openclosetime,daystoclose,openduetime,daysdue,missedduedate
0,2,101002444726,Intersection 3900 S GESSNER RD&10000 WESTPARK DR,Harris County,F,MID WEST,NULL,NaN,NaN,NaN,...,77063,2017,01,NaN,77063,0 days 00:09:53,0.006863,1 days 00:00:00,1.000000,0
1,6,12091836-101002444730,"3303 SAGE, HOUSTON TX 77056",HARRIS,G,GREATER UPTOWN,0451400060009,NaN,NaN,NaN,...,NaN,2017,01,77056,77056,32 days 12:49:18,32.534236,9 days 23:59:59,9.999988,1
2,9,101002444733,Intersection 1400 CAROLINE ST&1300 CLAY ST,Harris County,I,DOWNTOWN,NULL,NaN,NaN,NaN,...,77010,2017,01,NaN,77010,0 days 03:29:24,0.145417,1 days 00:00:00,1.000000,0
3,12,12091839-101002444736,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,0710210010015,NE,NW,MONDAY,...,NaN,2017,01,77022,77022,3 days 05:37:27,3.234340,0 days 23:59:58,0.999977,1
4,13,12091840-101002444737,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,0420050000055,NE,NW,MONDAY,...,NaN,2017,01,77022,77022,2 days 22:16:31,2.928137,2 days 00:00:00,2.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657188,1159867,101003785782,"3614 S MACGREGOR, HOUSTON TX 77021",HARRIS,D,MACGREGOR,0611350550011,SE,SW,TUESDAY,...,NaN,2019,12,77021,77021,1 days 19:05:30,1.795486,5 days 09:31:13,5.396678,0
657189,1159868,101003785783,"3547 TAMPA, HOUSTON TX 77021",HARRIS,D,MACGREGOR,0741210010025,SE,SW,TUESDAY,...,NaN,2019,12,77021,77021,1 days 18:49:53,1.784641,5 days 09:15:17,5.385613,0
657190,1159870,101003785785,"3415 WENTWORTH, HOUSTON TX 77004",HARRIS,D,MACGREGOR,0700280060004,SE,SW,TUESDAY,...,NaN,2019,12,77004,77004,1 days 18:22:03,1.765313,5 days 08:47:10,5.366088,0
657191,1159871,20024521-101003785786,"11035 AVON BROOK, HOUSTON TX 77034",HARRIS,E,SOUTH BELT / ELLINGTON,1319170030001,SE,SE,THURSDAY,...,NaN,2019,12,77034,77034,0 days 10:10:54,0.424236,10 days 00:00:00,10.000000,0


In [13]:
#Display null values in dataset
topdata.isnull().sum()

index                       0
case number                 0
sr location                 0
county                   1968
district                 2798
neighborhood             3755
tax id                   1958
trash quad             106569
recycle quad           108718
trash day              106569
heavy trash day        107179
recycle day            108716
key map                     0
management district    380775
department                  0
division                    0
sr type                     0
queue                       0
sla                         0
status                      0
sr create date              0
due date                    6
date closed              6803
overdue                  4982
title                       0
x                           0
y                           0
latitude                    0
longitude                   0
channel type                0
calczip                594941
year                        0
month                       0
truezip   

In [14]:
missingzip=topdata[topdata.zipcode.isnull()]
missingzip=missingzip[['latitude','longitude']].reset_index()
missingzip.drop_duplicates(inplace=True)
#zipmiss=missingzip[0:5000]
missingzip['calczip']=np.vectorize(zipinfo)(missingzip['latitude'].values,missingzip['longitude'].values)

In [15]:
newnullzip=nullzip.append([missingzip],sort=False)
newnullzip.drop(['index'],axis=1,inplace=True)
newnullzip.drop_duplicates(inplace=True)
newnullzip

,latitude,longitude,calczip
0,29.722099,-95.539078,77063
1,29.752398,-95.364693,77010
2,29.813465,-95.369751,77022
3,29.795120,-95.545807,77080
4,29.833042,-95.482864,77092
...,...,...,...
3487,29.854492,-95.468086,77092
3501,30.054660,-95.196577,77339
3525,29.711504,-95.337372,77021
3530,29.832123,-95.515526,77080


In [16]:
newnullzip.to_csv('../Clean Data Files/311latlngzipcodes.csv',index=False,header=True)

In [17]:
data311.to_csv("../../311_data_17_19.csv", index = False)

In [18]:
data311.shape

(1059994, 39)

In [19]:
data311WithZip = data311[data311['zipcode'].str.len() == 5]

In [20]:
#data2017WithZipA = data2017WithZip[data2017WithZip.sla.apply(lambda x: x.isnumeric())]
data311WithZip = data311WithZip[pd.to_numeric(data311WithZip['sla'], errors='coerce').notnull()]
data311WithZip = data311WithZip[pd.to_numeric(data311WithZip['overdue'], errors='coerce').notnull()]
data311WithZip['sla'] = data311WithZip['sla'].astype(float)
data311WithZip['overdue'] = data311WithZip['overdue'].astype(float)


In [28]:
data311WithZip['date_field_str'] = data311WithZip['sr create date'].dt.strftime('%Y%m%d')
data311WithZip['date_field'] = data311WithZip['sr create date'].dt.date 


In [48]:
data311WithZip['serv_type'] = data311WithZip['sr type']
data311WithZip['date_month'] = data311WithZip['sr create date'].dt.strftime('%b')
data311WithZip['yr'] = data311WithZip['sr create date'].dt.strftime('%Y')


In [70]:
#data311WithZip['yr'] = data311WithZip['sr create date'].dt.strftime('%Y')


In [71]:
dataAgg = data311WithZip.groupby(['zipcode', 'date_field', 'date_field_str', 'date_month', 'yr', 'serv_type', 'neighborhood']).agg(
    # Get max of the duration column for each group
    avg_sla=('sla', 'mean'),
    count_issues=('title', 'count'),
    # Get min of the duration column for each group
    avg_overdue=('overdue', 'mean'))  

In [72]:
dataAgg['avg_time'] = dataAgg['avg_sla'] +  dataAgg['avg_overdue']


In [73]:
dataAgg.to_csv("../static/data/agg_hist_311_data.csv")

In [55]:
dataAgg.reset_index(inplace=True)


In [53]:
weather_data_selected = pd.read_csv("../static/data/selected_weather_data_for_visual.csv") 

In [54]:
weather_data_selected

,date_field,date_field_str,tempMax,tempAvg,tempMin,dewMax,dewAvg,dewMin,humMax,humAvg,humMin,presMax,presAvg,presMin,precipitation
0,2017-01-01,20170101,77,64.3,55,51,40.6,30,80,46.9,19,27.9,27.9,27.8,0.00
1,2017-01-02,20170102,69,61.0,53,55,47.4,34,78,61.6,45,27.9,27.8,27.8,0.00
2,2017-01-03,20170103,73,62.5,54,51,46.4,42,78,57.8,34,27.9,27.8,27.8,0.00
3,2017-01-04,20170104,56,52.3,48,52,48.1,42,93,85.7,77,28.2,27.9,27.8,0.73
4,2017-01-05,20170105,75,55.8,40,47,41.3,36,86,62.9,24,28.4,28.3,28.1,1.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,2020-03-23,20200323,78,66.3,59,61,58.5,56,93,77.7,54,28.0,28.0,27.9,0.00
1049,2020-03-24,20200324,82,65.3,54,62,49.7,34,100,67.0,18,28.0,27.9,27.9,0.00
1050,2020-03-25,20200325,92,69.5,46,61,46.4,38,77,49.6,18,27.9,27.9,27.8,0.00
1051,2020-03-26,20200326,90,76.5,64,67,63.6,57,90,66.2,45,27.9,27.8,27.7,0.00


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [59]:
dataAgg.dtypes

zipcode            object
date_field         object
date_field_str     object
date_month         object
serv_type          object
neighborhood       object
avg_sla           float64
count_issues        int64
avg_overdue       float64
avg_time          float64
dtype: object

In [62]:
weather_data_selected.dtypes

date_field         object
date_field_str     object
tempMax             int64
tempAvg           float64
tempMin             int64
dewMax              int64
dewAvg            float64
dewMin              int64
humMax              int64
humAvg            float64
humMin              int64
presMax           float64
presAvg           float64
presMin           float64
precipitation     float64
dtype: object

In [61]:
weather_data_selected.date_field_str = weather_data_selected.date_field_str.astype(str)

In [64]:

# dataAgg = dataAgg.merge(weather_data_selected, how="left", on="date_field_str")

In [65]:
# dataAgg.to_csv("../static/data/agg_hist_311_data.csv")

In [66]:
dataAgg.dtypes

zipcode            object
date_field_x       object
date_field_str     object
date_month         object
serv_type          object
neighborhood       object
avg_sla           float64
count_issues        int64
avg_overdue       float64
avg_time          float64
date_field_y       object
tempMax           float64
tempAvg           float64
tempMin           float64
dewMax            float64
dewAvg            float64
dewMin            float64
humMax            float64
humAvg            float64
humMin            float64
presMax           float64
presAvg           float64
presMin           float64
precipitation     float64
dtype: object